In [1]:
import pandas as pd
import gzip
from sklearn.cross_validation import train_test_split
import numpy as np
from bs4 import BeautifulSoup
from six.moves import html_parser
import unicodedata
import HTMLParser
import re
from nltk.corpus import stopwords
from sklearn.externals import joblib

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  default = ''
  df = {}
  for d in parse(path):
    df[i] = d
    #df.loc[i] = [d.get('title',default),d.get('categories',default)]
    i += 1
    #if i==100000: break
  return pd.DataFrame.from_dict(df, orient='index')
  

df1 = getDF('meta_Clothing_Shoes_and_Jewelry.json.gz')


In [ ]:
#create pickle
#fileObject = open('./255_pickle/maindataframe.pickle','wb')
#joblib.dump(df1, fileObject)
#fileObject.close()

In [ ]:
#read code
#fileObject = open('./255_pickle/maindataframe.pickle','rb')
#df1 = joblib.load(fileObject)
#fileObject.close()

In [4]:
df_final = df1[['categories','title']]

In [ ]:
#create pickle
#fileObject = open('./255_pickle/df_final.pickle','wb')
#joblib.dump(df_final, fileObject)
#fileObject.close()

In [ ]:
#read code
#fileObject = open('./255_pickle/df_final.pickle','rb')
#df_final = joblib.load(fileObject)
#fileObject.close()

In [6]:
def  give_full_path(category,posn):
     try:
        return ( ' -> '.join((category)[posn]))
     except IndexError:
        return 'no-category'
     except TypeError:
        return 'no-category'

In [ ]:
df_final['categories1'] =  df_final['categories'].apply(give_full_path,posn=0)
df_final['categories2'] =  df_final['categories'].apply(give_full_path,posn=1)
df_final['categories3'] =  df_final['categories'].apply(give_full_path,posn=2)

In [8]:
##use this for demonstration
print (df_final.loc[4][['title'][0]])
print (df_final.loc[4][['categories1'][0]])
print (df_final.loc[4][['categories2'][0]])
print (df_final.loc[4][['categories3'][0]])
print (df_final.loc[1][['categories2','categories1','categories3']])
#print fullstring

Lantin White Visor Wrap Around Ski Style Aviator Sunglasses with Black Lenses
Clothing, Shoes & Jewelry -> Women -> Accessories -> Sunglasses & Eyewear Accessories -> Sunglasses
Clothing, Shoes & Jewelry -> Men -> Accessories -> Sunglasses & Eyewear Accessories -> Sunglasses
Clothing, Shoes & Jewelry -> Novelty, Costumes & More -> Band & Music Fan -> Accessories
categories2                                          no-category
categories1    Clothing, Shoes & Jewelry -> Girls -> Clothing...
categories3                                          no-category
Name: 1, dtype: object


In [ ]:
df_final.dropna(subset=['title'], inplace=True)

In [10]:
df_1st_category =df_final[(df_final['categories1'] != 'no-category')]

In [11]:
##retains deepest categories only for labels with single category 
category_path_df = df_1st_category.groupby('categories1').agg(np.size).reset_index()
category_path_df.sort_values(by='categories1', inplace=True)
category_path_df['category_path_next'] = category_path_df['categories1'].shift(-1)
category_path_list = []
for i, value in category_path_df.iterrows():
    category_path = value['categories1']
    category_path_next = value['category_path_next']
    if str(category_path) not in str(category_path_next):
        category_path_list.append(category_path)
category_path_df = pd.DataFrame(category_path_list, columns=['category1'])
df_1st_category = df_1st_category[df_1st_category['categories1'].isin(category_path_df['category1'])]

In [12]:
df_2nd_category =df_1st_category[(df_1st_category['categories2'] != 'no-category')]

In [13]:
##retains deepest categories only for labels with two categories
category_path_df = df_2nd_category.groupby('categories2').agg(np.size).reset_index()
category_path_df.sort_values(by='categories2', inplace=True)
category_path_df['category_path_next'] = category_path_df['categories2'].shift(-1)
category_path_list = []
for i, value in category_path_df.iterrows():
    category_path = value['categories2']
    category_path_next = value['category_path_next']
    if str(category_path) not in str(category_path_next):
        category_path_list.append(category_path)
category_path_df = pd.DataFrame(category_path_list, columns=['category2'])
df_2nd_category = df_2nd_category[df_2nd_category['categories2'].isin(category_path_df['category2'])]

In [14]:
df_3rd_category =df_2nd_category[df_2nd_category['categories3'] != 'no-category']

In [15]:
##retains deepest categories only for labels with three categories
category_path_df = df_3rd_category.groupby('categories3').agg(np.size).reset_index()
category_path_df.sort_values(by='categories3', inplace=True)
category_path_df['category_path_next'] = category_path_df['categories3'].shift(-1)
category_path_list = []
for i, value in category_path_df.iterrows():
    category_path = value['categories3']
    category_path_next = value['category_path_next']
    if str(category_path) not in str(category_path_next):
        category_path_list.append(category_path)
category_path_df = pd.DataFrame(category_path_list, columns=['category3'])
df_3rd_category = df_3rd_category[df_3rd_category['categories3'].isin(category_path_df['category3'])]

In [16]:
df_1st_category =df_1st_category[(df_1st_category['categories2'] == 'no-category') & (df_1st_category['categories3'] == 'no-category')].reset_index()
df_2nd_category =df_2nd_category[df_2nd_category['categories3'] == 'no-category'].reset_index()
df_3rd_category = df_3rd_category.reset_index()

In [17]:
print df_1st_category.shape
print df_2nd_category.shape
print df_3rd_category.shape

(450014, 6)
(213902, 6)
(349445, 6)


In [18]:
print df_3rd_category.dtypes

index           int64
categories     object
title          object
categories1    object
categories2    object
categories3    object
dtype: object


In [19]:
df_2nd_category.index = df_2nd_category.index + 450014

In [20]:
df_3rd_category.index = df_3rd_category.index + 450014+213902

In [21]:
print df_1st_category.loc[0]

index                                                          1
categories     [[Clothing, Shoes & Jewelry, Girls, Clothing, ...
title                                 Ballet Dress-Up Fairy Tutu
categories1    Clothing, Shoes & Jewelry -> Girls -> Clothing...
categories2                                          no-category
categories3                                          no-category
Name: 0, dtype: object


In [22]:
print df_2nd_category.loc[450014]

index                                                          3
categories     [[Clothing, Shoes & Jewelry, Women, Accessorie...
title          RiZ Women's Beautify Crafted &frac12; Rimmed F...
categories1    Clothing, Shoes & Jewelry -> Women -> Accessor...
categories2    Clothing, Shoes & Jewelry -> Novelty, Costumes...
categories3                                          no-category
Name: 450014, dtype: object


In [23]:
print df_3rd_category.loc[450014+213902]

index                                                          4
categories     [[Clothing, Shoes & Jewelry, Women, Accessorie...
title          Lantin White Visor Wrap Around Ski Style Aviat...
categories1    Clothing, Shoes & Jewelry -> Women -> Accessor...
categories2    Clothing, Shoes & Jewelry -> Men -> Accessorie...
categories3    Clothing, Shoes & Jewelry -> Novelty, Costumes...
Name: 663916, dtype: object


In [24]:
type (df_3rd_category.loc[450014+213902]['categories3'])

str

In [25]:
combined_categories_data = pd.concat([df_1st_category,df_2nd_category,df_3rd_category])

In [26]:
def enc_title(cat_title):
    try:
        encoded_title = unicodedata.normalize('NFKD', unicode(cat_title, 'utf-8', 'ignore')).encode('ascii', 'ignore')
        encoded_title = HTMLParser.HTMLParser().unescape(encoded_title).encode('ascii', 'ignore')
        encoded_title = encoded_title.lower()
        excluded='-/.%'
        encoded_title=re.split("[^" + excluded + "\w]+",encoded_title)
        word_set= [token for token in  encoded_title if not token.isdigit()]
        stop = set(stopwords.words('english'))
        word_set = [w for w in word_set if not w in stop]
        word_set = [w for w in word_set if len(w) > 2]
        encoded_title=(" ".join(word_set))
        
    except TypeError:  
        encoded_title = 'NA'

    return encoded_title

In [27]:
combined_categories_data['title'] =  combined_categories_data['title'].apply(enc_title)

In [28]:
combined_categories_data['combined_categories']= combined_categories_data[['categories1', 'categories2','categories3']].apply(lambda x: '<br>'.join(str(value) if str(value)!='no-category' else '<br>'  for value in x), axis=1)
                                                 


In [29]:
print combined_categories_data.dtypes

index                   int64
categories             object
title                  object
categories1            object
categories2            object
categories3            object
combined_categories    object
dtype: object


In [30]:
pd.options.display.max_colwidth = 1000

In [31]:
print combined_categories_data.loc[700000][['categories1', 'categories2','categories3']]

categories1    Clothing, Shoes & Jewelry -> Shoes & Accessories: International Shipping Available
categories2                                              Clothing, Shoes & Jewelry -> D -> Diesel
categories3                         Clothing, Shoes & Jewelry -> Men -> Shoes -> Fashion Sneakers
Name: 700000, dtype: object


In [32]:
print combined_categories_data.loc[700000]['combined_categories']

Clothing, Shoes & Jewelry -> Shoes & Accessories: International Shipping Available<br>Clothing, Shoes & Jewelry -> D -> Diesel<br>Clothing, Shoes & Jewelry -> Men -> Shoes -> Fashion Sneakers


In [ ]:
#fileObject = open('./255_pickle/combined_categories_data.pickle','wb')
#joblib.dump(combined_categories_data, fileObject)
#fileObject.close()

In [6]:
##read code
fileObject = open('./255_pickle/combined_categories_data.pickle','rb')
combined_categories_data = joblib.load(fileObject)
fileObject.close()

In [7]:
df = combined_categories_data.sample(frac=.1).reset_index(drop=True)

In [8]:
df.shape

(101336, 7)

In [9]:
#fileObject = open('./255_pickle/df.pickle','wb')
#joblib.dump(df, fileObject)
#fileObject.close()

In [2]:
##read code
from sklearn.externals import joblib
fileObject = open('./255_pickle/df.pickle','rb')
df = joblib.load(fileObject)
fileObject.close()

In [3]:
df_small = df.sample(frac=0.3).reset_index(drop=True)

In [4]:
#fileObject = open('./255_pickle/df_small.pickle','wb')
#joblib.dump(df_small, fileObject)
#fileObject.close()

In [2]:
##read code
from sklearn.externals import joblib
fileObject = open('./255_pickle/df_small.pickle','rb')
df_small = joblib.load(fileObject)
fileObject.close()

In [3]:
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
lin_clf = svm.LinearSVC()
train_x, test_x, train_y, test_y = train_test_split(df_small.title, df_small.combined_categories, test_size = 0.2, random_state = 42)
vectorizer = TfidfVectorizer(norm = 'l2', use_idf = True,max_df=0.001, smooth_idf = False, sublinear_tf = True,ngram_range=(1,2))
train_trans = vectorizer.fit_transform(train_x)
train_trans = train_trans.toarray()

In [4]:
train_trans.shape

(24320, 110632)

In [37]:
#write vectorizer
#fileObject = open('./255_pickle/vectorizer.pickle','wb')
#joblib.dump(vectorizer, fileObject)
#fileObject.close()

In [ ]:
##read vectorizer
#fileObject = open('./255_pickle/vectorizer.pickle','rb')
#vectorizer = joblib.load(fileObject)
#fileObject.close()

In [ ]:
#from sklearn.linear_model import SGDClassifier
#clf = SGDClassifier(loss="hinge", penalty="l2")
#clf.fit(train_trans, train_y)

/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_clf = RandomForestClassifier()
random_clf.fit(train_trans,train_y)

In [6]:
lin_clf.fit(train_trans,train_y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [7]:
fileObject = open('./255_pickle/lin_clf.pickle','wb')
joblib.dump(lin_clf, fileObject)
fileObject.close()

In [ ]:
#fileObject = open('./255_pickle/lin_clf.pickle','rb')
#lin_clf = joblib.load(fileObject)
#fileObject.close()

In [30]:
test_string = ['sports shoes']
test_string = np.array(test_string)
test_vector = vectorizer.transform(test_string).toarray()

In [31]:
test_vector.shape

(1, 110632)

In [32]:
lin_clf.predict(test_vector)[0]

'Clothing, Shoes & Jewelry -> Women -> Shoes -> Sandals<br><br><br><br>'

In [33]:
test_for_acc = vectorizer.transform(test_x).toarray()

In [34]:
from sklearn.metrics import accuracy_score
y_pred = lin_clf.predict(test_for_acc)

In [35]:
accuracy_score(test_y, y_pred)

0.31507975661897714

In [36]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, y_pred)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])